In [77]:
import glob
import pandas as pd
import email
from email import policy
from src.parser.MailTextParser import parse_mime_tree
from src.parser.MailSubjectParser import parse_mime_subject
from src.parser.MailSenderParser import parse_mime_sender
from typing import Dict, List, Optional, Tuple, Union
import re

## TODO

1. Mail Distribution YEAR, MONTH
2. SUBJECT ANALYSIS
3. SENDER ANALYSIS (cleaning first)
4. TEXT ANALYSIS (cleaning first)

## Helper stuff


In [2]:
MONTH_MAPPING = {
    "01": "JAN",
    "02": "FEB",
    "03": "MAR",
    "04": "APR",
    "05": "MAI",
    "06": "JUN",
    "07": "JUL",
    "08": "AUG",
    "09": "SEP",
    "10": "OCT",
    "11": "NOV",
    "12": "DEC"
}

In [49]:
def guess_encoding(file_path):
    encodings = ['utf-8', 'iso-8859-1', 'windows-1252']  # Add more as needed
    for enc in encodings:
        try:
            with open(file_path, encoding=enc) as file:
                content = file.read()
                if content == None:
                    print("NONE")
                return enc
        except UnicodeDecodeError:
            continue
    print(f"Could not open {file_path}. Maybe wrong encoding?")
    return None

## MAIN FUNCTIONS

In [52]:
def get_emails_from_folder(year_range: Tuple[int,int]) -> Dict[int, Dict[str, Dict[int, email.message.Message]]]:
    '''Get all emails from email folders stored under data/.

        Parameters
        ----------
            year_range: Tuple(int,int)
                The year range, where starting year is _inclusive_ and end year is _exclusive_.

        Returns
        -------
            (Dict) 
            
            {
                YEAR: {
                    MONTH: {
                        1: <email.message.Message>
                    }
                    ...
                }
            }
    '''
    emails = dict()
    counter = 0
    for year in range(year_range[0],year_range[1]):
        emails[year] = dict()
        for month in MONTH_MAPPING:
            emails[year][month] = dict()
            for idx, file in enumerate(glob.glob(f"data/{year}/yggdrasill_{year}-{month}/*")):
                # guess encoding first, since there are quite a few non-UTF-8 mails
                enc = guess_encoding(file)
                with open(file, "r", encoding=enc) as f:
                    emails[year][month][idx] = email.message_from_file(f, policy=policy.default) # the policy-default is very important for subject titles
                    # check if message has been parsed correctly
                    if emails[year][month][idx].keys() == []:
                        print(f"Issue with mail!")
                        print(file)                            
                counter += 1
    print(f"Received {counter} emails from {year_range[0]} to {year_range[1]-1}.") 
    return emails

In [69]:
emails = get_emails_from_folder((1997,2000))

Received 598 emails from 1997 to 1999.


In [58]:
## save emails
import pickle

with open("storage.pkl", "wb") as f:
    pickle.dump(emails, f)

In [7]:
import pickle

with open("storage.pkl", "rb") as f:
    emails = pickle.load(f)

In [8]:
#emails

In [59]:
def get_content_from_emails(email_dict: Dict[int, Dict[str, Dict[int, email.message.Message]]]) -> Dict:
    '''Extract CONTENT, SUBJECT, SENDER from emails retrieved via get_emails_from_folder().

        Parameters
        ----------
            email_dict: Dict[str, Dict[str, Dict[int, email.message.Message]]
                The dictionary with emails, retrieved using get_emails_from_folder().

        Returns
        -------
            (Dict) 
            
            {
                YEAR: {
                    MONTH: {
                        1: {
                            "subject": "<SUBJECT>",
                            "text": "<TEXT>",
                            "sender": "<SENDER>"
                        }
                        ...
                        "counter": <INT>
                    }
                    ...
                    "counter": <INT>
                }
                "counter": <INT>
            }
    '''
    emails = dict()
    counter = 0
    for year in email_dict:
        counter_year = 0
        emails[year] = dict()
        for month in email_dict[year]:
            counter_month = 0
            emails[year][month] = dict()
            for idx, mail in email_dict[year][month].items():
                text = parse_mime_tree(mail)
                sender = parse_mime_sender(mail)
                subject = parse_mime_subject(mail)
                emails[year][month][idx] = {
                    "text": text,
                    "subject": subject,
                    "sender": sender
                }
                # incrementing counters
                counter += 1
                counter_year += 1
                counter_month += 1
            emails[year][month]["counter"] = counter_month
        emails[year]["counter"] = counter_year
    emails["counter"] = counter
    return emails

In [70]:
content = get_content_from_emails(emails)

/mnt/c/Users/pyris/OneDrive/Dokumente/Projekte/Artikel/2024_Yggdrasil_Artikel/src/parser/MailTextParser.py:42: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  return BeautifulSoup(content, "html.parser").get_text() # removing html tags and noise


In [71]:
content[1999]["counter"]

127

In [72]:
# sanity check
sum_ = 0
for y in content:
    if type(y) == int:
        for m in content[y]:
            if m != "counter":
                sum_ += content[y][m]["counter"]

assert sum_ == content["counter"]

## Testing

### Cleaning "subject" field

In [73]:
clean_subject = re.compile(r"^re:\s+\[ygg\]|^fwd:\s+\[ygg\]|^aw:\s+\[ygg\]|^re:\s+fwd:\s+\[ygg\]|^\[ygg\]|\baw:|\bfwd:|\bfw:|\bre:", re.IGNORECASE)

In [76]:
for idx,mail in content[1997]["12"].items():
    if type(idx) != str:
        print("=====================================")
        if mail["subject"] == None:
            print(mail)
        else:
            subject = re.sub(clean_subject, "", mail["subject"])
            subject = subject.strip().lower()
            print(subject)
        

anfrage
hagiographie
(weitergel.) tagung ueber neue relgionen
endangered religions and new species


In [97]:
def clean_subject(subject: str) -> str:
    '''Function to clean an email-subject using RE.
    '''
    CLEANING_PATTERN = re.compile(r"^re:\s+\[ygg\]|^fwd:\s+\[ygg\]|^aw:\s+\[ygg\]|^re:\s+fwd:\s+\[ygg\]|^\[ygg\]|\baw:|\bfwd:|\bfw:|\bre:", re.IGNORECASE)
    subject = re.sub(CLEANING_PATTERN, " ", subject)
    subject = re.sub(r"\s{2,}", " ", subject)
    subject = subject.strip().lower()
    return subject

In [111]:
## get titles related to conferences and cfps

def get_specific_subjects(emails_in, pattern) -> Dict[int, Dict[str,Union[List,int]]]:
    '''Function to count and collect all subjects matching a RE pattern.

        Parameters
        ----------
            emails_in: Dict[]
                The nested content dict from get_content_from_emails().
            pattern: re
                The RE to match.

        Returns
        -------
            Dictionary:
            {
                <YEAR>: {
                    "titles": List[str],
                    "counter": int
                }
            }
    '''
    cfp_dict = dict()
    for year,months in emails_in.items():
        if type(year) == int:
            cfp_dict[year] = {
                "titles": list(),
                "counter": 0
            }
            for month,messages in months.items():
                if month != "counter":
                    for _,msg in messages.items():
                        if _ != "counter":
                            cln_subject = clean_subject(msg["subject"]) if msg["subject"] else ""
                            if re.match(pattern, cln_subject):
                                cfp_dict[year]["titles"].append(msg["subject"])
                                cfp_dict[year]["counter"] += 1
    return cfp_dict
                    
                

In [112]:
CFP_RE_PATTERN = re.compile(r"cfa|cfp|konferenz|conference|call for paper(s)?")

get_specific_subjects(content, CFP_RE_PATTERN)

{1997: {'titles': [], 'counter': 0},
 1998: {'titles': ['Conference at Greifswald University August 31 - September 4, 1998'],
  'counter': 1},
 1999: {'titles': [], 'counter': 0}}